# ECG データ可視化

## 概要

ECG データの基本的な統計と可視化を簡潔に行います。

- **患者データ**: 年齢・性別分布
- **ECG 解析**: 心拍数・QRS・QT 間隔分布
- **波形データ**: サンプル波形表示


In [ ]:
# ライブラリインポート
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import japanize_matplotlib

warnings.filterwarnings("ignore")
# plt.style.use("default")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 10

print("ライブラリ読み込み完了")

In [ ]:
# データ読み込み
DATA_DIR = Path("./data")

try:
    df_patients = pd.read_parquet(DATA_DIR / "patient_data.parquet")
    df_ecg = pd.read_parquet(DATA_DIR / "ecg_analysis.parquet")
    print(f"患者データ: {len(df_patients):,} 件")
    print(f"ECGデータ: {len(df_ecg):,} 件")
except Exception as e:
    print(f"データ読み込みエラー: {e}")

In [ ]:
display(df_patients.head())
display(df_ecg.head())

In [ ]:
# データ基本情報と欠損値分析
print("=== 患者データ基本情報 ===")
print(df_patients.info())
print("\n=== ECGデータ基本情報 ===")
print(df_ecg.info())

# 欠損値の割合を可視化
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 患者データの欠損値割合
missing_patients = (df_patients.isnull().sum() / len(df_patients) * 100).sort_values(
    ascending=False
)
missing_patients = missing_patients[missing_patients > 0]
if len(missing_patients) > 0:
    missing_patients.plot(kind="bar", ax=axes[0], color="coral")
    axes[0].set_title("患者データ - 欠損値割合 (%)")
    axes[0].set_ylabel("欠損値割合 (%)")
    axes[0].tick_params(axis="x", rotation=45)
else:
    axes[0].text(
        0.5, 0.5, "欠損値なし", ha="center", va="center", transform=axes[0].transAxes
    )
    axes[0].set_title("患者データ - 欠損値割合 (%)")

# ECGデータの欠損値割合
missing_ecg = (df_ecg.isnull().sum() / len(df_ecg) * 100).sort_values(ascending=False)
missing_ecg = missing_ecg[missing_ecg > 0]
if len(missing_ecg) > 0:
    missing_ecg.plot(kind="bar", ax=axes[1], color="lightcoral")
    axes[1].set_title("ECGデータ - 欠損値割合 (%)")
    axes[1].set_ylabel("欠損値割合 (%)")
    axes[1].tick_params(axis="x", rotation=45)
else:
    axes[1].text(
        0.5, 0.5, "欠損値なし", ha="center", va="center", transform=axes[1].transAxes
    )
    axes[1].set_title("ECGデータ - 欠損値割合 (%)")

plt.tight_layout()
plt.show()

## 患者属性分析


In [ ]:
# 患者属性グラフ
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 年齢分布
age_data = df_patients["age"].dropna()
axes[0].hist(age_data, bins=20, alpha=0.7, color="skyblue", edgecolor="black")
axes[0].set_title("年齢分布")
axes[0].set_xlabel("年齢")
axes[0].set_ylabel("患者数")
axes[0].axvline(
    age_data.mean(), color="red", linestyle="--", label=f"平均: {age_data.mean():.1f}歳"
)
axes[0].legend()

# 性別分布
gender_counts = df_patients["gender"].value_counts()
axes[1].pie(gender_counts.values, labels=gender_counts.index, autopct="%1.1f%%")
axes[1].set_title("性別分布")

# 年齢層別性別分布
df_patients["age_group"] = pd.cut(
    df_patients["age"],
    bins=[0, 30, 50, 70, 100],
    labels=["~30", "30-50", "50-70", "70~"],
)
age_gender = pd.crosstab(df_patients["age_group"], df_patients["gender"])
age_gender.plot(kind="bar", ax=axes[2])
axes[2].set_title("年齢層別性別分布")
axes[2].tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

print(f"総患者数: {len(df_patients):,} 人")
print(f"平均年齢: {age_data.mean():.1f}歳")

## ECG 測定値分析


In [ ]:
# ECG主要指標グラフ
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 心拍数分布
hr_data = df_ecg["heart_rate"].dropna()
axes[0, 0].hist(hr_data, bins=30, alpha=0.7, color="red", edgecolor="black")
axes[0, 0].axvline(
    hr_data.mean(),
    color="black",
    linestyle="--",
    label=f"平均: {hr_data.mean():.1f} bpm",
)
axes[0, 0].set_title("心拍数分布")
axes[0, 0].set_xlabel("心拍数 (bpm)")
axes[0, 0].legend()

# QRS幅分布
qrs_data = df_ecg["qrs_duration"].dropna()
axes[0, 1].hist(qrs_data, bins=30, alpha=0.7, color="blue", edgecolor="black")
axes[0, 1].axvline(
    qrs_data.mean(),
    color="black",
    linestyle="--",
    label=f"平均: {qrs_data.mean():.1f} ms",
)
axes[0, 1].set_title("QRS幅分布")
axes[0, 1].set_xlabel("QRS幅 (ms)")
axes[0, 1].legend()

# QT間隔分布
qt_data = df_ecg["qt_interval"].dropna()
axes[1, 0].hist(qt_data, bins=30, alpha=0.7, color="green", edgecolor="black")
axes[1, 0].axvline(
    qt_data.mean(),
    color="black",
    linestyle="--",
    label=f"平均: {qt_data.mean():.1f} ms",
)
axes[1, 0].set_title("QT間隔分布")
axes[1, 0].set_xlabel("QT間隔 (ms)")
axes[1, 0].legend()

# PR間隔分布
pr_data = df_ecg["pr_interval"].dropna()
axes[1, 1].hist(pr_data, bins=30, alpha=0.7, color="orange", edgecolor="black")
axes[1, 1].axvline(
    pr_data.mean(),
    color="black",
    linestyle="--",
    label=f"平均: {pr_data.mean():.1f} ms",
)
axes[1, 1].set_title("PR間隔分布")
axes[1, 1].set_xlabel("PR間隔 (ms)")
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# 統計サマリー
print("ECG測定値統計:")
print(f"心拍数: {hr_data.mean():.1f} ± {hr_data.std():.1f} bpm")
print(f"QRS幅: {qrs_data.mean():.1f} ± {qrs_data.std():.1f} ms")
print(f"QT間隔: {qt_data.mean():.1f} ± {qt_data.std():.1f} ms")
print(f"PR間隔: {pr_data.mean():.1f} ± {pr_data.std():.1f} ms")

In [ ]:
# ミネソタコード分析
print("ミネソタコード分析")

# ミネソタコードの分布を取得
minnesota_data = df_ecg["minnesota_code"].dropna()
print(f"ミネソタコード記録数: {len(minnesota_data)}")
print(f"ユニークコード数: {minnesota_data.nunique()}")

# 上位ミネソタコードを取得（頻度順）
top_minnesota = minnesota_data.value_counts().head(15)

# 可視化（3つのグラフに変更、1つのグラフあたりの横幅を広げる）
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
fig.suptitle("ミネソタコード分布分析", fontsize=16, fontweight="bold")

# 1. 上位ミネソタコードの頻度（棒グラフ）
top_minnesota.plot(kind="bar", ax=axes[0], color="steelblue", alpha=0.8)
axes[0].set_title("上位15ミネソタコード頻度")
axes[0].set_xlabel("ミネソタコード")
axes[0].set_ylabel("頻度")
axes[0].tick_params(axis="x", rotation=45)

# 2. 主要コードカテゴリ分析（最初の数字で分類）
main_categories = minnesota_data.str.extract(r"^(\d+)")[0]
main_cat_counts = main_categories.value_counts().head(10)
axes[1].pie(main_cat_counts.values, labels=main_cat_counts.index, autopct="%1.1f%%")
axes[1].set_title("主要コードカテゴリ分布")

# 3. ミネソタコード有無の分布
has_minnesota = df_ecg["minnesota_code"].notna()
minnesota_status = ["コードあり", "コードなし"]
minnesota_counts = [has_minnesota.sum(), (~has_minnesota).sum()]

axes[2].bar(
    minnesota_status, minnesota_counts, color=["lightcoral", "lightblue"], alpha=0.8
)
axes[2].set_title("ミネソタコード記録状況")
axes[2].set_ylabel("患者数")

# 各バーの上に数値を表示
for i, count in enumerate(minnesota_counts):
    axes[2].text(
        i, count + max(minnesota_counts) * 0.01, f"{count:,}", ha="center", va="bottom"
    )

plt.tight_layout()
plt.show()

# 統計サマリー出力
print("\nミネソタコード統計:")
print(f"総患者数: {len(df_ecg):,}")
print(
    f"ミネソタコード記録あり: {has_minnesota.sum():,} ({100*has_minnesota.mean():.1f}%)"
)
print(
    f"ミネソタコード記録なし: {(~has_minnesota).sum():,} ({100*(~has_minnesota).mean():.1f}%)"
)
print(f"最も多いコード: {top_minnesota.index[0]} ({top_minnesota.iloc[0]}件)")


print("\n上位5ミネソタコード:")
for code, count in top_minnesota.head().items():
    print(f"  {code}: {count}件 ({100*count/len(minnesota_data):.1f}%)")

## 波形データサンプル


In [ ]:
# 波形データ読み込み（12誘導表示）
try:
    df_waveform = pd.read_parquet(DATA_DIR / "waveform_data.parquet")

    # 1つの患者のデータを取得
    first_patient = df_waveform["recording_id"].iloc[0]
    patient_waves = df_waveform[df_waveform["recording_id"] == first_patient]

    # 利用可能な誘導を取得（最大12個）
    available_leads_raw = patient_waves["lead_name"].unique()
    available_leads = [lead.strip('"') for lead in available_leads_raw]
    display_leads = sorted(available_leads)[:12]  # 最大12誘導

    # 表示設定
    fig, axes = plt.subplots(4, 3, figsize=(16, 12))
    fig.suptitle(f"12誘導ECG波形 - {first_patient}", fontsize=16, fontweight="bold")

    # 元の誘導名とクリーンな誘導名のマッピング
    lead_mapping = {lead.strip('"'): lead for lead in available_leads_raw}

    # 各誘導を表示
    for i, lead in enumerate(display_leads):
        row, col = divmod(i, 3)
        ax = axes[row, col]

        # データを取得
        original_lead_name = lead_mapping.get(lead, lead)
        lead_data = patient_waves[patient_waves["lead_name"] == original_lead_name]
        lead_data = lead_data.sort_values("sequence_number")

        if len(lead_data) > 0:
            ax.plot(
                lead_data["sequence_number"],
                lead_data["value"],
                linewidth=1,
                color="blue",
            )
            ax.set_title(f"誘導 {lead}", fontsize=12)
        else:
            ax.text(
                0.5,
                0.5,
                f"誘導 {lead}\nデータなし",
                ha="center",
                va="center",
                transform=ax.transAxes,
            )

        ax.set_xlabel("時間")
        ax.set_ylabel("振幅")
        ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"波形データ表示完了 - 患者ID: {first_patient}, 誘導数: {len(display_leads)}")

except Exception as e:
    print(f"波形データエラー: {e}")